In [49]:
import requests
import json

#Json part

'''Get the url'''
params = (
    ('detail', 'contributions'),
)

response = requests.get('https://indico.cern.ch/export/event/867789.json', params=params)


'''Change to JSON'''
Jsonresponse = response.json()

'''Get into folder'''
Jsonresponse=Jsonresponse["results"][0]["contributions"]

'''To save the correct folder when it is found'''
result=None


'''Find the folder with the right id'''
for item in Jsonresponse:
    result=item
    if result["id"]=="409":
        break;

#Fix for multiple files?
# Find the file and the path to download the file
data_file_names=result["folders"][0]["attachments"][0]["filename"]
file_download_path=result["folders"][0]["attachments"][0]["download_url"]

description=result["description"]
Title=result["title"]
Upload_type=None

#Fix for multiple authors
Author_Names=result["primaryauthors"][0]["fullName"]
Author_Affiliation=result["primaryauthors"][0]["affiliation"]


if type(data_file_names)!=str:
    print("file name must be string")
if type(file_download_path)!=str:
    print("file download path must be string")
if type(description)!=str:
    print("description must be string")


#Zenodo part
#Fix so user can choose upload type
Uploadtype="nan"

# Access token to Zenodo
Access_Token="Something"
if type(Access_Token)!=str:
    print("file name must be string")


headers = {"Content-Type": "application/json"}


#create preliminary zenodo entry
r=requests.post('https://sandbox.zenodo.org/api/deposit/depositions', 
                params={'access_token': Access_Token}, json={},headers=headers)

deposition_id= r.json()['id']

# Gives your uploaded file the correct name
data={'name': data_file_name}

#File to upload
files={'file': open(file_download_path, 'rb')}

#Put in file
r=requests.post('https://sandbox.zenodo.org/api/deposit/depositions/%s/files' % deposition_id, 
                params={'access_token': Access_Token}, data=data, files=files)

#All data about the entry
data={ 'metadata': {'title': Title, 'upload_type': 'poster', 'description': description,
                    'creators': [{'name': Author_Names, 'affiliation': Author_Affiliation}]}}

#Insert data into entry
r=requests.put('https://sandbox.zenodo.org/api/deposit/depositions/%s' % deposition_id, 
               params={'access_token': Access_Token}, data=json.dumps(data), headers=headers)


KeyError: 'id'